In [1]:
# pre-requisites:
# ensure that the keylines folder (https://sdk.keylines.com/public/keylines-6.5.0-64488.zip or your preferred version)
# is in ~/.ipython/nbextensions
#jupyter notebook   --NotebookApp.allow_origin='https://colab.research.google.com'   --port=8888   --NotebookApp.port_retries=0

In [1]:
import networkx as nx
import json
from IPython.display import Javascript
import pandas as pd
import random

In [10]:
# Load excel ELT into pandas DFs
multi_sheet_file = pd.ExcelFile("C:\\Dev\\Chorus\\chorus-mockup\\src\\assets\\data\\resolved_combined.xlsx")

# Gets the sheet names in the file
excel_sheet_names = multi_sheet_file.sheet_names

dict_of_sheets = {}
for sheet in excel_sheet_names:
    dict_of_sheets[sheet] = pd.read_excel(multi_sheet_file, sheet_name=sheet)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Dev\\Chorus\\chorus-mockup\\src\\assets\\data\\resolved_combined.xlsx'

In [ ]:
# Load into networkx
row_record_type = "PATHWAY REF"
graph = nx.Graph()

icon_map = {
    "Suspect": {"t": '👤', "c": "blue"},
    "Address": {"t": '🏢', "c": "grey"},
    "Location": {"t": '🏢', "c": "grey"},
}

label_map = {
    "Suspect": ["First Name", "Surname", "dob"],
    "Address": ["Line address1", "Line Address2", "County", "Postcode"]
}

def handle_entity_row(sheet_name, row):
    label = row['Label']
    
#     if sheet_name in label_map:
#         label_cols = label_map[sheet_name]
#         for col in label_cols:
#             label += str(row[col]) + " "
#     else:
#         label = str(pd.Series.to_csv(row))

    if sheet_name in icon_map:
        icon_attr = icon_map[sheet_name]
    else:
        icon_attr = {"t": '❓', "c": "red"}
    
    graph.add_node(row[sheet_name + " ID"], label=row['Label'], icon_attr=icon_attr)

def handle_link_row(sheet_name, row):
    
    graph.add_edge(row[0], row[1])
    
for sheet_name, dataframe in dict_of_sheets.items():
    pk_row_name = row_record_type + " ID"
    if ' to ' in sheet_name:
        # link sheet
        dataframe.apply(lambda row: handle_link_row(sheet_name, row), axis=1)
    else:
        # entity sheet
        dataframe.apply(lambda row: handle_entity_row(sheet_name, row), axis=1)

graph_json_data = nx.node_link_data(graph)

In [ ]:
# parse data into something keylines can handle
import json 

keylines_data = {
    "type": "LinkChart",
    "items": []
}

for node in graph_json_data["nodes"]:
    if 'label' not in node.keys():
        print(node)
        continue
    label = node['label']

        
    keylines_data['items'].append({
        "id": node['id'],
        "type": "node",
        "t": label,
        "fi": node['icon_attr'] if 'icon_attr' in node else {"t": '❓', "c": "red"}
    })
    
for link in graph_json_data["links"]:
    keylines_data['items'].append({
        "id": str(link['source']) + "-" + str(link['target']),
        "type": "link",
        "id1": link['source'],
        "id2": link['target']
    })
    

    
Javascript("""
          window.data={};
          """.format(json.dumps(keylines_data)))


In [ ]:
%%javascript
$('#kl').remove();
element.append('<div id="kl"style="height:600px"></div>');

In [ ]:
%%javascript

function linkedToOpenCombo(linkId, instance) {
      const link = instance.getItem(linkId);
      const node1 = instance.combo().find(link.id1);
      const node2 = instance.combo().find(link.id2);
      // Filter out non-combo ends of the link
      const linkEndCombos = [node1, node2].filter((id) => instance.combo().isCombo(id));
      // Return true only if one of the links is an open combo
      return linkEndCombos.some((comboId) => iInstance.combo().isOpen(comboId));
}

require(["nbextensions/keylines/js/keylines"], function (KeyLines) {
    // We can use the KeyLines API as normal in here
    KeyLines.promisify();
    KeyLines.create({type: 'chart', id: 'kl'}).then(chart => {
    
        chart.load(window.data).then(() => {
            chart.zoom('fit');
            const engine = window.KeyLines.getGraphEngine();
            chart.on('selection-change', () => {
      // Get the current state
      const ids = chart.selection().filter((id) => chart.getItem(id).type === 'node');
      if (ids.length === 1) {
        // Get, show (if required), and foreground the neighbours of the underlying items
        engine.load(chart.serialize());
        // The method for getting neighbours of combos and noncombos is slightly different
        // So we have to use a different method for each
        const combos = ids.filter((id) => chart.combo().isCombo(id));
        const comboNeighbours = chart.graph().neighbours(combos);
        const nonCombos = ids.filter((id) => !chart.combo().isCombo(id));
        const nonComboNeighbours = engine.neighbours(nonCombos);
        // Get all the links involved
        const links = comboNeighbours.links.concat(nonComboNeighbours.links);
        // And filter them such that they only include ones linked to an open combo.
        chart.combo().reveal(links.filter(linkID => linkedToOpenCombo(linkID, chart)));
        if (combos.length) {
          // This allows combos to become selected as there's no 'all' option for chart.foreground
          chart.foreground((item) => links.includes(item.id), { type: 'link', items: 'toplevel' });
        }
        if (nonCombos.length) {
          chart.foreground((item) => links.includes(item.id), { type: 'link' });
        }
      } else {
        // No items selected, foreground everything
        chart.combo().reveal([]);
        chart.foreground(() => true);
      }
            })
        }).then(() => {
            chart.layout("radial");
        })
    })
});